In [2]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.8 MB/s eta 0:00:00


In [3]:
from pyngrok import ngrok
ngrok.set_auth_token("358J96RoaekkhqIFd55Z1OihaFr_6rczzfHtYKSxHEWFs4D9F")


In [4]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import load_model
from io import BytesIO
from PIL import Image

#Prediction function
def predict_and_explain(model, image_array, layer_name, img_size=128):
  #load and preprocess the image
    img_resized = cv2.resize(image_array, (img_size, img_size))
    img_norm = img_resized / 255.0
    input_img = np.expand_dims(img_norm, axis=0)

    #prediction of the mask
    pred_mask = model.predict(input_img)
    pred_mask = (pred_mask > 0.5).astype(np.uint8)
    pred_mask_vis = np.squeeze(pred_mask)

    #Integrating GRADE cam
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_img)
        loss = tf.reduce_mean(predictions)
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    cam = np.zeros(conv_outputs.shape[:2], dtype=np.float32)
    for i, w in enumerate(pooled_grads):
        cam += w * conv_outputs[:, :, i]
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)

    #applying heatmap overlay
    cam_resized = cv2.resize(cam, (img_size, img_size))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    overlay = np.uint8(0.6 * heatmap + 0.4 * (img_resized * 255))

    return img_resized, pred_mask_vis, overlay



#Streamlite UI
st.set_page_config(page_title="MRI Segmentation + GradCAM", layout="wide")

# Custom CSS
st.markdown("""
    <style>
    .main {
        background-color: #f8fafc;
        font-family: 'Segoe UI', sans-serif;
    }
    .stButton>button {
        background-color: #2b6cb0;
        color: white;
        font-weight: bold;
        border-radius: 10px;
        height: 3em;
        width: 100%;
        border: none;
    }
    .stButton>button:hover {
        background-color: #1e4e8c;
        color: white;
    }
    img {
        border-radius: 10px;
    }
    </style>
""", unsafe_allow_html=True)

# Header Section
st.markdown("## Brain MRI Segmentation with Explainable AI (Grad-CAM)")
st.markdown("""
**This model used to segment out the tumour region from the MRI Image**
and highlights the **most influential regions** using **Grad-CAM heatmaps**
""")

st.divider()

# Image upload
uploaded = st.file_uploader("Upload an MRI Image", type=["png", "jpg", "jpeg","tif"])

if uploaded:
    file_bytes = np.asarray(bytearray(uploaded.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    st.image(image, caption="Uploaded MRI (Preview)", width=300)

    st.markdown("#### Ready to analyze this image?")
    if st.button("Run"):
        with st.spinner("Running model... generating segmentation and Grad-CAM..."):
            model = load_model("/content/drive/MyDrive/project_datasets/segmented_model(50epoch).h5", compile=False)
            last_conv_layer = "conv2d_10"
            input_img, mask, overlay = predict_and_explain(model, image, last_conv_layer)
        st.success("Prediction complete!")

        st.divider()

        # Display results
        st.markdown("### Model Results")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.image(input_img, caption="Input MRI", use_container_width=True)
        with col2:
            st.image(mask * 255, caption="Predicted Mask", use_container_width=True)
        with col3:
            st.image(overlay, caption="Grad-CAM Explanation", use_container_width=True)

        st.divider()

        # Download buttons
        mask_img = Image.fromarray((mask * 255).astype(np.uint8))
        overlay_img = Image.fromarray(overlay)

        buf1, buf2 = BytesIO(), BytesIO()
        mask_img.save(buf1, format="PNG")
        overlay_img.save(buf2, format="PNG")

        st.download_button("Download Predicted Mask", data=buf1.getvalue(),
                           file_name="predicted_mask.png", mime="image/png")

        st.download_button("Download Grad-CAM Heatmap", data=buf2.getvalue(),
                           file_name="gradcam_heatmap.png", mime="image/png")


Writing app.py


In [5]:
!nohup streamlit run app.py --server.port 8501 &>/dev/null&

In [6]:
from pyngrok import ngrok
public_url = ngrok.connect(addr="8501")
print("Streamlit App Live at:")
print(public_url.public_url)


Streamlit App Live at:
https://unlimned-maryalice-caboched.ngrok-free.dev
